<a href="https://colab.research.google.com/github/IamRam3/dry-fruit-detection-segmentation/blob/main/test_modularity_(dryfruits)_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detection

In [ ]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
import torch.utils.data
import torchvision
from torchvision.transforms import ToTensor
from pycocotools.cocoeval import COCOeval
import json
import numpy as np
from tqdm import tqdm
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
!git clone https://github.com/IamRam3/dry-fruit-detection-segmentation.git

fatal: destination path 'dry-fruit-detection-segmentation' already exists and is not an empty directory.


In [ ]:
!mv dry-fruit-detection-segmentation dry_fruit_detection_segmentation

mv: cannot move 'dry-fruit-detection-segmentation' to 'dry_fruit_detection_segmentation/dry-fruit-detection-segmentation': Directory not empty


In [ ]:
#!rm -rf dry_fruit_detection_segmentation/

In [ ]:
from dry_fruit_detection_segmentation.src.data_setup import CocoDataset, coco_dataset_dir, CustomDataLoader
from dry_fruit_detection_segmentation.utils import get_transform, convert_to_coco_format, get_detection_model, train_pipe, create_video_from_images

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
batch_size=4

In [ ]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-classification.v1i.coco.zip -d /content/DryFruits-classification.v1i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-classification.v1i.coco.zip
replace /content/DryFruits-classification.v1i.coco/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Detection.v2i.coco.zip -d /content/DryFruits-Detection.v2i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Detection.v2i.coco.zip
replace /content/DryFruits-Detection.v2i.coco/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
dataset_path = "/content/DryFruits-Detection.v2i.coco"

In [ ]:
dataset = coco_dataset_dir(dataset_path)

In [ ]:
train_dataset, val_dataset, test_dataset = dataset.create_custom_datasets(ToTensor)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
train_data_loader, val_data_loader, test_data_loader = CustomDataLoader(train_dataset, val_dataset, test_dataset, batch_size).create_data_loader()

In [ ]:
from dry_fruit_detection_segmentation.utils import get_detection_model

In [ ]:
# extra class; for background
num_classes = len(train_dataset.coco.getCatIds()) + 1
model = get_detection_model(num_classes, custom_focal_loss=True)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 100

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
best_map = 0.0
epochs_no_improve = 0
patience = 5  # stop if no improvement for 5 consecutive epochs
model_name = "detection_v2"

In [ ]:
train_pipe(num_epochs, train_data_loader, model, val_data_loader, convert_to_coco_format, dataset
, device, optimizer, model_name, patience=5, best_map=0.0, segmentation=False)

Training Epoch 1: 100%|██████████| 53/53 [00:40<00:00,  1.31it/s]


[Epoch 1] Training Loss: 0.2949


Validating: 100%|██████████| 5/5 [00:01<00:00,  2.79it/s]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.02s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.450
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.712
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.460
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.579
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.643
 Average Recall     (AR) @[ IoU=0.50:0.95 |

Training Epoch 2: 100%|██████████| 53/53 [00:40<00:00,  1.31it/s]


...

Removing existing prediction file: coco_eval/predictions_False.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.02s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.733
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.922
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.922
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.733
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.772
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all

Training Epoch 15: 100%|██████████| 53/53 [00:40<00:00,  1.31it/s]


[Epoch 15] Training Loss: 0.0315


Validating: 100%|██████████| 5/5 [00:01<00:00,  2.81it/s]

Removing existing prediction file: coco_eval/predictions_False.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.737
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.909
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.835
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.737
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.783
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all

In [ ]:
create_video_from_images(dataset.test_dir, "test_video.mp4", fps=1)

Video saved to test_video.mp4


In [ ]:
model_save_path = f"best_{model_name}_model.pth"
save_path = os.path.join("saved_models", model_save_path)
model = get_detection_model(num_classes, custom_focal_loss=True)
model.load_state_dict(torch.load(save_path))
model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
id_to_name = {
    1: "almond",
    2: "cashew",
    3: "grapes",
    4: "raisin"
}

In [ ]:
from dry_fruit_detection_segmentation.utils import save_predicted_video, save_predicted_image

In [ ]:
save_predicted_image(dataset.test_dir+"/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.98746b3d6b7cd7d53599adf17435b3ff.jpg", "detection_out1.jpg", model, device, id_to_name, threshold=0.4)

[✓] Inference time: 0.1218 seconds


In [ ]:
save_predicted_video("test_video.mp4", "detection_V2_out.mp4", model, device, id_to_name, threshold=0.4)

[✓] Saved 10 annotated frames to video: detection_V2_out.mp4


In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 97.9 MB/s eta 0:00:00


In [ ]:
from dry_fruit_detection_segmentation.utils import FasterRCNNWrapper

In [ ]:
wrapped_model = FasterRCNNWrapper(model)
wrapped_model.cpu()
wrapped_model.eval()

dummy_input = torch.randn(1, 3, 640, 640)  # still on CPU

# export
torch.onnx.export(
    wrapped_model,
    dummy_input,
    "detection_v1.onnx",
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["boxes", "labels", "scores"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "boxes": {0: "batch_size"},
        "labels": {0: "batch_size"},
        "scores": {0: "batch_size"},
    }
)

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4624: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  * torch.tensor(scale_factors[i], dtype=torch.float32)
/usr/local/lib/python3.11/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.11/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_y = torch.min(boxes_y, torch.tensor(height, dty

## MobileNet model

In [ ]:
model = get_detection_model(num_classes, mobilenet=True, custom_focal_loss=True)
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 100

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100%|██████████| 74.2M/74.2M [00:00<00:00, 78.2MB/s]


In [ ]:
best_map = 0.0
epochs_no_improve = 0
patience = 5  # stop if no improvement for 5 consecutive epochs
model_name = "mobilenet_v2"

In [ ]:
train_pipe(num_epochs, train_data_loader, model, val_data_loader, convert_to_coco_format, dataset
, device, optimizer, model_name, patience=5, best_map=0.0, segmentation=False)

Training Epoch 1: 100%|██████████| 53/53 [00:07<00:00,  6.85it/s]


[Epoch 1] Training Loss: 1.6053


Validating: 100%|██████████| 5/5 [00:00<00:00, 12.20it/s]


Removing existing prediction file: coco_eval/predictions_False.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.370
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.661
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.370
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all

Training Epoch 2: 100%|██████████| 53/53 [00:07<00:00,  7.31it/s]


...

Removing existing prediction file: coco_eval/predictions_False.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.668
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.805
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.805
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.668
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.760
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all

Training Epoch 23: 100%|██████████| 53/53 [00:07<00:00,  7.29it/s]


[Epoch 23] Training Loss: 0.3591


Validating: 100%|██████████| 5/5 [00:00<00:00, 12.32it/s]

Removing existing prediction file: coco_eval/predictions_False.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.657
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.818
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.794
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.657
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.761
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all

In [ ]:
model_save_path = f"best_{model_name}_model.pth"
save_path = os.path.join("saved_models", model_save_path)
model = get_detection_model(num_classes, mobilenet=True, custom_focal_loss=True)
model.load_state_dict(torch.load(save_path))
model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block):

In [ ]:
save_predicted_image(dataset.test_dir+"/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.98746b3d6b7cd7d53599adf17435b3ff.jpg", "mobilenetv2_out1.jpg", model, device, id_to_name, threshold=0.4)

[✓] Inference time: 0.0250 seconds


In [ ]:
save_predicted_video("test_video.mp4", "mobilenet_V2_out.mp4", model, device, id_to_name, torch.threshold=0.4)

#Segmentation

In [ ]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Segmentation.v2i.coco.zip -d /content/DryFruits-Segmentation.v2i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Segmentation.v2i.coco.zip
  inflating: /content/DryFruits-Segmentation.v2i.coco/README.dataset.txt  
  inflating: /content/DryFruits-Segmentation.v2i.coco/README.roboflow.txt  
   creating: /content/DryFruits-Segmentation.v2i.coco/test/
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.ee5a89bca40b5620557c06690a160e4c.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_806_jpg.rf.de00fdc33a36d4aed2d2b3354c47ed7c.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/ALMOND_SANORA_ALMOND_SANORA_292_jpg.rf.089d18d7179d9d70c86f32037cafe374.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/ALMOND_SANORA_ALMOND_SANORA_905_jpg.rf.822d8eb52e60f587dfb543e358580f32.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/CASHEW_SPECIAL_CASHEW_SPECIAL_552_jpg.rf.62b04a

In [ ]:
dataset_path = "/content/DryFruits-Segmentation.v2i.coco"

In [ ]:
dataset = coco_dataset_dir(dataset_path)

In [ ]:
train_dataset, val_dataset, test_dataset = dataset.create_custom_datasets(ToTensor, segmentation=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
train_data_loader, val_data_loader, test_data_loader = CustomDataLoader(train_dataset, val_dataset, test_dataset, batch_size).create_data_loader()

In [ ]:
from dry_fruit_detection_segmentation.utils import get_segmentation_model

In [ ]:
# extra class; for background
num_classes = len(train_dataset.coco.getCatIds()) + 1
model = get_segmentation_model(num_classes, custom_focal_loss=True)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 100

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
best_map = 0.0
epochs_no_improve = 0
patience = 5  # stop if no improvement for 5 consecutive epochs
model_name = "segmentation_v2"

In [ ]:
train_pipe(num_epochs, train_data_loader, model, val_data_loader, convert_to_coco_format, dataset
, device, optimizer, model_name, patience=5, best_map=0.0, segmentation=True)

Training Epoch 1:   0%|          | 0/53 [00:00<?, ?it/s]/content/dry_fruit_detection_segmentation/src/data_setup.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  masks = torch.as_tensor(masks, dtype=torch.uint8)
Training Epoch 1: 100%|██████████| 53/53 [00:57<00:00,  1.08s/it]


[Epoch 1] Training Loss: 0.6049


Validating: 100%|██████████| 5/5 [00:03<00:00,  1.49it/s]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.02s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.367
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.136
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.339
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.488
 Average Recall     (AR) @[ IoU=0.50:0.95 |

Training Epoch 2: 100%|██████████| 53/53 [00:57<00:00,  1.09s/it]


...

Removing existing prediction file: coco_eval/predictions_True.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.715
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.814
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.737
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.727
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.833
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all 

Training Epoch 18: 100%|██████████| 53/53 [00:57<00:00,  1.08s/it]


[Epoch 18] Training Loss: 0.0670


Validating: 100%|██████████| 5/5 [00:03<00:00,  1.54it/s]

Removing existing prediction file: coco_eval/predictions_True.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.02s).
Validation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.676
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.790
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.713
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.709
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.818
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all 

In [ ]:
create_video_from_images(dataset.test_dir, "object_segmentation_in.mp4", fps=1)

In [ ]:
model_save_path = f"best_{model_name}_model.pth"
save_path = os.path.join("saved_models", model_save_path)
model = get_segmentation_model(num_classes, custom_focal_loss=False)
model.load_state_dict(torch.load(save_path))
model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
example_img = "/content/DryFruits-Segmentation.v2i.coco/test/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.ee5a89bca40b5620557c06690a160e4c.jpg"

In [ ]:
basic_colors = {
    'red': (255, 0, 0),
    'green': (0, 128, 0),
    'blue': (0, 0, 255),
    'yellow': (255, 255, 0),
    #'cyan': (0, 255, 255),
    #'magenta': (255, 0, 255),
    'black': (0, 0, 0),
    #'white': (255, 255, 255),
    'gray': (128, 128, 128),
    'orange': (255, 165, 0),
    #'brown': (165, 42, 42),
    'purple': (128, 0, 128),
}

In [ ]:
from dry_fruit_detection_segmentation.utils import closest_basic_color_name, detect_shape, save_segmented_image, save_segmented_video

In [ ]:
save_segmented_image(example_img, "segment_v2_output.jpg", model, device, id_to_name, basic_colors)

[✓] Inference time: 0.1228 seconds
[✓] Saved segmented image to: segment_v2_output.jpg


/content/dry_fruit_detection_segmentation/utils.py:432: RuntimeWarning: overflow encountered in scalar subtract
  dist = sum((rc - cc) ** 2 for rc, cc in zip(requested_color, rgb))
/content/dry_fruit_detection_segmentation/utils.py:432: RuntimeWarning: overflow encountered in scalar add
  dist = sum((rc - cc) ** 2 for rc, cc in zip(requested_color, rgb))


In [ ]:
save_segmented_video("object_segmentation_in.mp4", "object_segmentation_out.mp4", model, device, id_to_name, basic_colors, threshold=0.4)

[✓] Inference time: 0.1267 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00000.jpg
[✓] Inference time: 0.1230 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00001.jpg
[✓] Inference time: 0.0816 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00002.jpg
[✓] Inference time: 0.0815 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00003.jpg
[✓] Inference time: 0.0803 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00004.jpg
[✓] Inference time: 0.0773 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00005.jpg
[✓] Inference time: 0.0826 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00006.jpg
[✓] Inference time: 0.0836 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00007.jpg
[✓] Inference time: 0.0830 seconds
[✓] Saved segmented image to: object_segmentation_out.mp4/frame_00008.jpg
[✓] Inference time:

In [ ]:
from dry_fruit_detection_segmentation.utils import MaskRCNNWrapper

In [ ]:
wrapped_model = MaskRCNNWrapper(model)
wrapped_model.cpu()
wrapped_model.eval()

dummy_input = torch.randn(1, 3, 640, 640)  # still on CPU

# export
torch.onnx.export(
    wrapped_model,
    dummy_input,
    "segmentation_v1.onnx",
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["boxes", "labels", "scores", "masks"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "boxes": {0: "batch_size"},
        "labels": {0: "batch_size"},
        "scores": {0: "batch_size"},
        "masks": {0: "batch_size"},
    }
)

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4624: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  * torch.tensor(scale_factors[i], dtype=torch.float32)
/usr/local/lib/python3.11/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.11/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_y = torch.min(boxes_y, torch.tensor(height, dty